In [1]:
import pandas as pd
import numpy as np
import math

In [2]:


# Re-defining constants and parameters
n_simulations = 1000
min_bid = 900
max_bid = 1000
bid_increment = 1
resale_value = 1000

# Corrected function for calculating expected profit based on the updated strategy
def calculate_updated_expected_profit(lower_bid, higher_bid, reserve_prices, resale_value):
    """
    Calculates the expected profit given a lower bid and a higher bid, offered sequentially in the correct order.
    """
    # Sellers that accept the lower bid
    accept_lower_bid = reserve_prices[reserve_prices <= lower_bid]
    
    # Sellers that do not accept the lower bid but accept the higher bid
    accept_higher_bid = reserve_prices[(reserve_prices > lower_bid) & (reserve_prices <= higher_bid)]
    
    # Calculate total profit
    profit_from_lower_bid = len(accept_lower_bid) * (resale_value - lower_bid)
    profit_from_higher_bid = len(accept_higher_bid) * (resale_value - higher_bid)
    
    total_profit = profit_from_lower_bid + profit_from_higher_bid
    return total_profit

# Re-running the simulation with the strategy of offering a lower bid first, then a higher bid
res = []

for i in range(n_simulations):
    reserve_prices = np.random.triangular(left=min_bid, mode=max_bid, right=max_bid, size=10000)
    max_profit = 0
    optimal_lower_bid = 0
    optimal_higher_bid = 0
    for lower_bid in range(min_bid, max_bid, bid_increment):
        for higher_bid in range(lower_bid + bid_increment, max_bid + bid_increment, bid_increment):
            total_profit = calculate_updated_expected_profit(lower_bid, higher_bid, reserve_prices, resale_value)
            if total_profit > max_profit:
                max_profit = total_profit
                optimal_lower_bid = lower_bid
                optimal_higher_bid = higher_bid
        
        res.append([optimal_lower_bid, optimal_higher_bid, max_profit])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/yj/jjmvh7dd6sx95p4gzltx4j2r0000gn/T/ipykernel_12709/2101488248.py", line None, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1326, in structured_traceback
    return VerboseTB.structured_t

In [19]:
print(np.mean([x[0] for x in res]))
print(np.mean([x[1] for x in res]))
print(np.mean([x[2] for x in res]))
'''
938.18828
974.20059
189797.77504
'''

938.26
974.24763
189615.89716


'\n938.18828\n974.20059\n189797.77504\n'

In [22]:
starfruit_cache = {
        "vwap": 1,
        "vwap_-1": 1,
        "vwap_-2": 1,
        "vwap_change": 1,
        "imbalance_volume_signed": 1,
        "best_mean": 1,
        "best_mean_-1": 1,
        "best_mean_-2": 1,
    }

None in starfruit_cache.values()

False

In [4]:
# Re-define the exchange rates as the execution state has been reset.
exchange_rates = {
    'Pizza Slice': {'Wasabi Root': 0.48, 'Snowball': 1.52, 'Shells': 0.71},
    'Wasabi Root': {'Pizza Slice': 2.05, 'Snowball': 3.26, 'Shells': 1.56},
    'Snowball': {'Pizza Slice': 0.64, 'Wasabi Root': 0.3, 'Shells': 0.46},
    'Shells': {'Pizza Slice': 1.41, 'Wasabi Root': 0.61, 'Snowball': 2.08}
}

# Initial capital in Shells
initial_capital = 2000000

# Brute force approach to find the best trading strategy
# We are looking for the best path: Shells -> A -> B -> C -> Shells
# where A, B, C are the three other currencies, and we visit each one exactly once.

best_sequence = None
max_shells = 0

# All possible sequences of the three currencies excluding the starting and ending SeaShells
from itertools import permutations

# Calculate the value after the sequence of trades
for sequence in permutations(exchange_rates.keys() - {'Shells'}):
    # Start with the initial capital in Shells
    current_capital = initial_capital
    
    # Start trading with Shells
    current_currency = 'Shells'
    
    # Go through the selected sequence of trades
    for next_currency in sequence:
        # Update the capital based on the exchange rate
        current_capital *= exchange_rates[current_currency][next_currency]
        # Update the current currency
        current_currency = next_currency
    
    # After finishing the sequence, trade back to Shells
    current_capital *= exchange_rates[current_currency]['Shells']
    
    # Check if this sequence gives us more Shells than the previous best
    if current_capital > max_shells:
        max_shells = current_capital
        best_sequence = sequence

best_sequence, max_shells


(('Pizza Slice', 'Wasabi Root', 'Snowball'), 2029858.56)

In [14]:
from itertools import permutations

# Define the exchange rates from the image provided
exchange_rates = {
    'Pizza Slice': {'Pizza Slice': 1, 'Wasabi Root': 0.48, 'Snowball': 1.52, 'Shells': 0.71},
    'Wasabi Root': {'Pizza Slice': 2.05, 'Wasabi Root': 1, 'Snowball': 3.26, 'Shells': 1.56},
    'Snowball': {'Pizza Slice': 0.64, 'Wasabi Root': 0.3, 'Snowball': 1, 'Shells': 0.46},
    'Shells': {'Pizza Slice': 1.41, 'Wasabi Root': 0.61, 'Snowball': 2.08, 'Shells': 1}
}

# Initial capital in Shells
initial_capital = 2000000

# Function to calculate the profit for a given path
def calculate_profit(path, initial_capital, exchange_rates):
    # print(path)
    capital = initial_capital
    for i in range(len(path) - 1):
        capital *= exchange_rates[path[i]][path[i+1]]
    return capital

# Find the optimal trading path
def find_optimal_path(exchange_rates, initial_capital):
    max_profit = 0
    optimal_path = []

    paths, profits = [], []

    # Generate all permutations for the three currencies
    for perm in permutations(['Pizza Slice', 'Wasabi Root', 'Snowball']):
        # Add the Shells to start and end
        perm = ('Shells',) + perm + ('Shells',)

        # Insert the additional trade in each possible position
        for i in range(1, len(perm) - 1):  # Exclude the first and last position (Shells)
            for extra in ['Pizza Slice', 'Wasabi Root', 'Snowball']:
                # Create a new path with the extra trade inserted
                new_path = perm[:i] + (extra,) + perm[i:]
                # Calculate the profit for this path
                profit = calculate_profit(new_path, initial_capital, exchange_rates)
                paths.append(new_path),
                profits.append(profit)
                # Update the optimal path if this is the best so far
                if profit > max_profit:
                    max_profit = profit
                    optimal_path = new_path

    return optimal_path, max_profit, paths, profits

# Find the optimal path
optimal_path, max_profit, paths, profits = find_optimal_path(exchange_rates, initial_capital)

# Print the result
print("Optimal path:", optimal_path)
print("Maximum profit in SeaShells:", max_profit)


Optimal path: ('Shells', 'Pizza Slice', 'Wasabi Root', 'Snowball', 'Wasabi Root', 'Shells')
Maximum profit in SeaShells: 2065160.448


In [15]:
def sort_list(list1, list2):
 
    zipped_pairs = zip(list2, list1)
 
    z = [[x, _] for _, x in sorted(zipped_pairs, reverse=True)]
 
    return z

res = sort_list(paths, profits)

for p, pr in res:
    print(p, pr, '\n')

('Shells', 'Pizza Slice', 'Wasabi Root', 'Snowball', 'Wasabi Root', 'Shells') 2065160.448 

('Shells', 'Pizza Slice', 'Snowball', 'Pizza Slice', 'Wasabi Root', 'Shells') 2054180.0447999998 

('Shells', 'Pizza Slice', 'Snowball', 'Pizza Slice', 'Wasabi Root', 'Shells') 2054180.0447999998 

('Shells', 'Pizza Slice', 'Wasabi Root', 'Wasabi Root', 'Snowball', 'Shells') 2029858.56 

('Shells', 'Pizza Slice', 'Wasabi Root', 'Wasabi Root', 'Snowball', 'Shells') 2029858.56 

('Shells', 'Pizza Slice', 'Wasabi Root', 'Snowball', 'Snowball', 'Shells') 2029858.56 

('Shells', 'Pizza Slice', 'Pizza Slice', 'Wasabi Root', 'Snowball', 'Shells') 2029858.56 

('Shells', 'Pizza Slice', 'Pizza Slice', 'Wasabi Root', 'Snowball', 'Shells') 2029858.56 

('Shells', 'Pizza Slice', 'Snowball', 'Wasabi Root', 'Wasabi Root', 'Shells') 2006035.2 

('Shells', 'Pizza Slice', 'Snowball', 'Snowball', 'Wasabi Root', 'Shells') 2006035.2 

('Shells', 'Pizza Slice', 'Snowball', 'Snowball', 'Wasabi Root', 'Shells') 200603

In [17]:
import itertools
from typing import List

prices = {
    'Pizza Slice': {'Pizza Slice': 1, 'Wasabi Root': 0.48, 'Snowball': 1.52, 'Shells': 0.71},
    'Wasabi Root': {'Pizza Slice': 2.05, 'Wasabi Root': 1, 'Snowball': 3.26, 'Shells': 1.56},
    'Snowball': {'Pizza Slice': 0.64, 'Wasabi Root': 0.3, 'Snowball': 1, 'Shells': 0.46},
    'Shells': {'Pizza Slice': 1.41, 'Wasabi Root': 0.61, 'Snowball': 2.08, 'Shells': 1}
}

goods = ["Pizza Slice", "Wasabi Root", "Snowball", "Shells"]


def calculate_profit(path, initial_capital, exchange_rates):
    # print(path)
    capital = initial_capital
    for i in range(len(path) - 1):
        capital *= exchange_rates[path[i]][path[i+1]]
    return capital

def calc_money(trades: List[str], amount: float) -> float:
    for i in range(0, len(trades) - 1):
        good_from = trades[i]
        good_to = trades[i + 1]
        amount *= prices[good_from][good_to]
    return amount


if __name__ == '__main__':
    start_amount = 2000000
    result = []
    all_combinations = list(itertools.product(goods, repeat=4)) + \
        list(itertools.product(goods, repeat=3)) + \
        list(itertools.product(goods, repeat=2)) + \
        list(itertools.product(goods, repeat=1))

    # Go through all the combinations and append their result
    for combination in all_combinations:
        trade = ["Shells"] + list(combination) + ["Shells"]
        result.append((calculate_profit(trade, start_amount, exchange_rates=prices), trade))

    result.sort(key=lambda x: (-x[0], len(x[1])))
    print("Final amount: {}\nAchieved with the combination of: {}".format(result[0][0], result[0][1]))

Final amount: 2113938.7776
Achieved with the combination of: ['Shells', 'Pizza Slice', 'Wasabi Root', 'Shells', 'Pizza Slice', 'Shells']


In [20]:
for pr, p in result:
    if p == ['Shells', 'Pizza Slice', 'Wasabi Root', 'Snowball', 'Wasabi Root', 'Shells']:
        print(pr)

2065160.448


In [10]:
import itertools
import numpy as np

# Initial setup
prices = {
    'G26': (24, 2), 'G27': (70, 4), 'G28': (41, 3), 'G29': (21, 2), 'G30': (60, 4),
    'H26': (47, 3), 'H27': (82, 5), 'H28': (87, 5), 'H29': (80, 5), 'H30': (35, 3),
    'I26': (73, 4), 'I27': (89, 5), 'I28': (100, 8), 'I29': (90, 7), 'I30': (17, 2),
    'J26': (77, 5), 'J27': (83, 5), 'J28': (85, 5), 'J29': (79, 5), 'J30': (55, 4),
    'K26': (12, 2), 'K27': (27, 3), 'K28': (52, 4), 'K29': (15, 2), 'K30': (30, 3)
}
base_treasure = 7500
total_players = 10000
expedition_costs = [0, 25000, 75000]

# Calculate profit for a single expedition
def calc_profit(spot, hunters_distribution, expedition_cost):
    multiplier, num_hunters = prices[spot]
    # total_hunters = num_hunters + (hunters_distribution[spot] * total_players)
    total_treasure = base_treasure * multiplier
    # print(spot, num_hunters ,  (hunters_distribution[spot] * 100))
    profit = total_treasure / (num_hunters  + hunters_distribution[spot] * 100) - expedition_cost
    return profit

# Simulate rounds
def simulate_rounds(prices, base_treasure, expedition_costs, total_players):
    # Initialize distribution of hunters uniformly for the first round
    hunters_distribution = {spot: 1/len(prices) for spot in prices}
    profits = []
    net_profits = []
    all_spots = list(prices.keys())

    for round_number in range(3):
        # Calculate profits for all spots
        round_profits = {spot: calc_profit(spot, hunters_distribution, expedition_costs[min(round_number, 1)])
                         for spot in all_spots}
        profits.append(round_profits)

        # Calculate net profits (total profits minus the cost)
        net_profit = {spot: profit - (expedition_costs[1] if round_number > 0 else 0)
                      for spot, profit in round_profits.items()}
        net_profits.append(net_profit)

        # Update hunters distribution based on profits
        total_profit = sum(round_profits.values())
        hunters_distribution = {spot: profit / total_profit for spot, profit in round_profits.items()}

        # The best spot for the current round
        best_spot = max(net_profit, key=net_profit.get)
        print(f"Round {round_number + 1}, Best Spot: {best_spot}, Profit: {net_profit[best_spot]}")

    return net_profits

# Run simulation
net_profits = simulate_rounds(prices, base_treasure, expedition_costs, total_players)

# Function to calculate total net profit for a given combination of expeditions
def calculate_total_net_profit(combination, net_profits, expedition_costs):
    total_net_profit = 0
    for i, spot in enumerate(combination):
        total_net_profit += net_profits[i].get(spot, 0) - (expedition_costs[i] if i > 0 else 0)
    return total_net_profit

# Generate all valid combinations and their total net profits
all_combinations = []

# Single expedition
all_combinations.extend([(r1,) for r1 in net_profits[0]])

# Two expeditions
all_combinations.extend([(r1, r2) for r1 in net_profits[0] for r2 in net_profits[1] if r1 != r2])

# Three expeditions
all_combinations.extend([(r1, r2, r3) for r1 in net_profits[0] for r2 in net_profits[1] for r3 in net_profits[2] if len({r1, r2, r3}) == 3])

# Calculate net profits for all combinations
combination_profits = [(combination, calculate_total_net_profit(combination, net_profits, expedition_costs)) for combination in all_combinations]

# Sort the combinations by total net profit
sorted_combinations = sorted(combination_profits, key=lambda x: x[1], reverse=True)

# Print the sorted combinations with their total net profits
for combo, profit in sorted_combinations:
    print(f"Combination: {combo}, Total Net Profit: {profit}")


Round 1, Best Spot: I27, Profit: 74166.66666666667
Round 2, Best Spot: I27, Profit: 11584.942062179616
Round 3, Best Spot: I27, Profit: 11902.994759232308
Combination: ('I27',), Total Net Profit: 74166.66666666667
Combination: ('H28',), Total Net Profit: 72500.0
Combination: ('J28',), Total Net Profit: 70833.33333333333
Combination: ('J27',), Total Net Profit: 69166.66666666667
Combination: ('I26',), Total Net Profit: 68437.5
Combination: ('H27',), Total Net Profit: 68333.33333333333
Combination: ('H29',), Total Net Profit: 66666.66666666667
Combination: ('J29',), Total Net Profit: 65833.33333333333
Combination: ('G27',), Total Net Profit: 65625.0
Combination: ('J26',), Total Net Profit: 64166.666666666664
Combination: ('I28',), Total Net Profit: 62500.0
Combination: ('I29',), Total Net Profit: 61363.63636363636
Combination: ('I27', 'H28'), Total Net Profit: 60105.363754120626
Combination: ('I27', 'J28'), Total Net Profit: 59443.08449024542
Combination: ('H28', 'I27'), Total Net Profit